In [4]:
%%time
import awswrangler as wr
import os
import boto3
import re
import copy
import time
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role
import pandas as pd

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 27.9 µs


In [5]:
df_pred_test = wr.s3.read_csv('s3://ads508team7/jump_start/pred_test/data.csv')
df_pred_valid = wr.s3.read_csv('s3://ads508team7/jump_start/pred_valid/data.csv')

In [6]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

def perform():
    cmtx = pd.DataFrame(
        confusion_matrix(df_pred_test['Target'], df_pred_test['PredBinary'], labels=[0,1]), 
        index=['true: {:}'.format(x) for x in [0,1]], 
        columns=['pred: {:}'.format(x) for x in [0,1]])  

    EvalTable = pd.DataFrame(columns = ['Accuracy','Sensitivity','Specificity','F1Measure'], index=['xgboost'])

    # Calculating Accuracy
    EvalTable.Accuracy = \
    (cmtx.iloc[0,0]+cmtx.iloc[1,1]) / sum(sum(cmtx.values))

    # Calculating Sensitivity    
    EvalTable.Sensitivity = cmtx.iloc[0,0]/(cmtx.iloc[0,0]+cmtx.iloc[0,1])

    # Calculating Specificity   
    EvalTable.Specificity = cmtx.iloc[1,1]/(cmtx.iloc[1,0]+cmtx.iloc[1,1])     

    # Calculating F1 Measure   
    EvalTable.F1Measure = \
      2 * ((EvalTable.Specificity * EvalTable.Sensitivity) / 
           (EvalTable.Specificity + EvalTable.Sensitivity)) 

    return EvalTable

best_cut_off_point = {}   
for cut_off_point in range(40, 70, 1):
    print('when cut off point is:', cut_off_point/100)
    df_pred_test['PredBinary'] = [1 if float(value) >= (cut_off_point/100) else 0 for value in df_pred_test['Pred'].values]
    print('\n')
    EvalTable = perform()
    print(EvalTable.F1Measure.values[0])
    best_cut_off_point[cut_off_point/100] = EvalTable.F1Measure.values[0]

when cut off point is: 0.4


0.40867227883748647
when cut off point is: 0.41


0.4215487133637254
when cut off point is: 0.42


0.43169434423782554
when cut off point is: 0.43


0.4358628513404396
when cut off point is: 0.44


0.4405779515561822
when cut off point is: 0.45


0.4471701960919179
when cut off point is: 0.46


0.4570252881837173
when cut off point is: 0.47


0.46756787645098774
when cut off point is: 0.48


0.48213494364698306
when cut off point is: 0.49


0.5006438092775731
when cut off point is: 0.5


0.5247106687339761
when cut off point is: 0.51


0.547945218726523
when cut off point is: 0.52


0.5683841857475029
when cut off point is: 0.53


0.5828393947445273
when cut off point is: 0.54


0.5889035743246036
when cut off point is: 0.55


0.583211765097477
when cut off point is: 0.56


0.5606976313689884
when cut off point is: 0.57


0.5184317910867023
when cut off point is: 0.58


0.45177346920377576
when cut off point is: 0.59


0.376239889838511
when cut off point i

In [8]:
cut_off_df = pd.DataFrame.from_dict(best_cut_off_point, orient='index',columns = ['f1_measure'])
best_val = cut_off_df.sort_values(by='f1_measure', ascending=False)
best_val.head()

,f1_measure
0.54,0.588904
0.55,0.583212
0.53,0.582839
0.52,0.568384
0.56,0.560698


In [14]:
best = best_val.index[0]
df_pred_test['PredBinary'] = [1 if float(value) >= best else 0 for value in df_pred_test['Pred'].values]
EvalTable = perform()
print(EvalTable)

         Accuracy  Sensitivity  Specificity  F1Measure
xgboost  0.589605     0.569263     0.609947   0.588904
